In [2]:
from math import pi

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from PIL import Image

import tensorflow as tf
from tensorflow.keras import Sequential, activations, losses, metrics, optimizers
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, Lambda, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Creating the `tf.data.Dataset`

In [3]:
angles, paths = [], []
train_angles, train_paths = [], []
test_angles, test_paths = [], []

with open("./driving_dataset/data.txt", "r") as f:
    for line in f.readlines():
        image, angle = line.split(" ")
        full_path = f"C:/Users/abhi0/Desktop/autopilot-pytorch/driving_dataset/{image}"
        paths.append(full_path)
        angles.append(float(angle) * (np.pi / 180))

# Splitting into train/test
total_len = len(paths)
split = int(0.8 * total_len)

train_paths = paths[:split]
train_angles = angles[:split]

test_paths = paths[split:]
test_angles = angles[split:]

## Train and test data Dataframes

In [4]:
train_df = pd.DataFrame({"image_paths": train_paths, "angles": train_angles})
test_df = pd.DataFrame({"image_paths": test_paths, "angles": test_angles})

In [5]:
train_df.head()

,image_paths,angles
0,C:/Users/abhi0/Desktop/autopilot-pytorch/drivi...,0.0
1,C:/Users/abhi0/Desktop/autopilot-pytorch/drivi...,0.0
2,C:/Users/abhi0/Desktop/autopilot-pytorch/drivi...,0.0
3,C:/Users/abhi0/Desktop/autopilot-pytorch/drivi...,0.0
4,C:/Users/abhi0/Desktop/autopilot-pytorch/drivi...,0.0


In [6]:
test_df.head()

,image_paths,angles
0,C:/Users/abhi0/Desktop/autopilot-pytorch/drivi...,-0.193557
1,C:/Users/abhi0/Desktop/autopilot-pytorch/drivi...,-0.188321
2,C:/Users/abhi0/Desktop/autopilot-pytorch/drivi...,-0.175929
3,C:/Users/abhi0/Desktop/autopilot-pytorch/drivi...,-0.163712
4,C:/Users/abhi0/Desktop/autopilot-pytorch/drivi...,-0.131947


## Datagenerators

In [7]:
train_data_gen = ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=90,
    validation_split=0.15
)
test_data_gen = ImageDataGenerator()

In [8]:
# Train and validation generators

train_gen = train_data_gen.flow_from_dataframe(
    dataframe=train_df,
    x_col="image_paths",
    y_col="angles",
    class_mode="raw",
    target_size=(66, 200),
    subset="training",
    batch_size=32,
)
val_gen = train_data_gen.flow_from_dataframe(
    dataframe=train_df,
    x_col="image_paths",
    y_col="angles",
    class_mode="raw",
    target_size=(66, 200),
    subset="validation",
    batch_size=32,
)

Found 30876 validated image filenames.
Found 5448 validated image filenames.


## tf Datasets

In [9]:
# Train and validation tf datasets

train_ds = tf.data.Dataset.from_generator(
    generator=lambda: train_gen,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 66, 200, 3], [None]),
)
val_ds = tf.data.Dataset.from_generator(
    generator=lambda: val_gen,
    output_types=(tf.float32, tf.float32),
    output_shapes=([None, 66, 200, 3], [None]),
)

In [11]:
train_steps_per_epoch = int(np.ceil(30876 / 32))
val_steps_per_epoch = int(np.ceil(5448 / 32))

## Nvidia's Model

In [12]:
model = Sequential(
    [
        Conv2D(24, kernel_size=(5, 5), strides=(2, 2), activation="elu"),
        Conv2D(36, kernel_size=(5, 5), strides=(2, 2), activation="elu"),
        Conv2D(48, kernel_size=(5, 5), strides=(2, 2), activation="elu"),
        Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation="elu"),
        Conv2D(64, kernel_size=(3, 3), strides=(1, 1), activation="elu"),
        Dropout(0.5),
        Flatten(),
        Dense(units=100, activation="elu"),
        Dense(units=50),
        Dense(units=10),
        Dense(units=1),
    ]
)

In [13]:
model.build(input_shape=(1, 66, 200, 3))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (1, 31, 98, 24)           1824      
_________________________________________________________________
conv2d_1 (Conv2D)            (1, 14, 47, 36)           21636     
_________________________________________________________________
conv2d_2 (Conv2D)            (1, 5, 22, 48)            43248     
_________________________________________________________________
conv2d_3 (Conv2D)            (1, 3, 20, 64)            27712     
_________________________________________________________________
conv2d_4 (Conv2D)            (1, 1, 18, 64)            36928     
_________________________________________________________________
dropout (Dropout)            (1, 1, 18, 64)            0         
_________________________________________________________________
flatten (Flatten)            (1, 1152)                 0

In [14]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=5e-5),
    loss=losses.MeanSquaredError(),
)

In [ ]:
hist = model.fit(
    train_gen,
    steps_per_epoch=train_steps_per_epoch,
    validation_data=val_gen,
    validation_steps=val_steps_per_epoch,
    epochs=30,
)